In [9]:
import pandas as pd
import numpy as np
import os, re
from typing import Union
from typing import Optional, Tuple

In [2]:
df_fd = pd.read_csv("ignore/fd_summary.csv")
df_fd.head()

,metric,avg,std,cv,notes,prob_name,domain,planner,config
0,translate,0.456744,0.018699,0.040940,NaN,prob15:,driverlog,fd,lmcut
1,scoping,3.628331,0.151413,0.041731,NaN,prob15:,driverlog,fd,lmcut
2,translate_and_scope,4.085075,0.162906,0.039878,NaN,prob15:,driverlog,fd,lmcut
3,plan_unscoped_time,4.440337,0.147840,0.033295,NaN,prob15:,driverlog,fd,lmcut
4,plan_scoped_time,3.080146,0.071122,0.023090,NaN,prob15:,driverlog,fd,lmcut


In [3]:
def clean_col_name(s: str) -> str:
    return s.replace("_"," ").title()

fd_colmap_manual = {
    "plan_unscoped_generated_nodes": "Generated Nodes (Unscoped)",
    "plan_unscoped_node_expansions": "Node Expansions (Unscoped)",
    "plan_scoped_generated_nodes": "Generated Nodes (Scoped)",
    "plan_scoped_node_expansions": "Node Expansions (Scoped)",
    "plan_unscoped_time": "Plan Time (Unscoped)",
    "plan_scoped_time": "Plan Time (Scoped)",
    "encoding_size": "Encoding Size",
    "total_unscoped_time": "Total Time (Unscoped)",
    "total_scoped_time": "Total Time (Scoped)"
}

fd_colmap = {c: clean_col_name(c) for c in df_fd["metric"].unique()}
fd_colmap.update(fd_colmap_manual)
# df_fd = df_fd.rename(columns=fd_colmap)
# df_fd
df_fd["metric"] = df_fd["metric"].replace(fd_colmap)
df_fd

,metric,avg,std,cv,notes,prob_name,domain,planner,config
0,Translate,0.456744,0.018699,0.040940,NaN,prob15:,driverlog,fd,lmcut
1,Scoping,3.628331,0.151413,0.041731,NaN,prob15:,driverlog,fd,lmcut
2,Translate And Scope,4.085075,0.162906,0.039878,NaN,prob15:,driverlog,fd,lmcut
3,Plan Time (Unscoped),4.440337,0.147840,0.033295,NaN,prob15:,driverlog,fd,lmcut
4,Plan Time (Scoped),3.080146,0.071122,0.023090,NaN,prob15:,driverlog,fd,lmcut
...,...,...,...,...,...,...,...,...,...
283,Generated Nodes (Unscoped),NaN,NaN,NaN,NaN,prob14:,zenotravel,fd,ms
284,Node Expansions (Unscoped),NaN,NaN,NaN,NaN,prob14:,zenotravel,fd,ms
285,Generated Nodes (Scoped),NaN,NaN,NaN,NaN,prob14:,zenotravel,fd,ms
286,Node Expansions (Scoped),NaN,NaN,NaN,NaN,prob14:,zenotravel,fd,ms


In [27]:
def get_table_str(df: pd.DataFrame, metric: str) -> str:
    a = df.loc[metric, "avg"]
    if isinstance(a, str):
        try:
            a = float(a)
        except:
            pass
    if isinstance(a, (float,)):
        a = str(round(a, 2))
    sd = df.loc[metric, "std"]
    if isinstance(sd, str):
        try:
            sd = float(sd)
        except:
            pass
    if isinstance(sd, float):
        sd = str(round(sd,2))
    return f"{a} \pm {sd}"

def get_scoped_unscoped_names(s: str):
    return s + " (Scoped)", s + " (Unscoped)"

def remove_gt(s: Union[str, float]) -> Tuple[Union[str, float], bool]:
    """Returns float, and whether it started with >"""
    if isinstance(s, float):
        return s, False
    elif isinstance(s, str):
        if len(s) > 0 and s[0] == ">":
            return float(s[1:]), True
        else:
            return float(s), False

def get_better_col(df: pd.DataFrame, metric: str) -> Optional[str]:
    metric_scoped, metric_unscoped = get_scoped_unscoped_names(metric)
    v_scoped, v_unscoped = df.loc[metric_scoped, "avg"], df.loc[metric_unscoped, "avg"]
    (v_scoped, gt_scoped) = remove_gt(v_scoped)
    (v_unscoped, gt_unscoped) = remove_gt(v_unscoped)
    # Handle NaN
    if np.isnan(v_scoped) and np.isnan(v_unscoped):
        return None
    if np.isnan(v_scoped) and not np.isnan(v_unscoped):
        return metric_unscoped
    if not np.isnan(v_scoped) and np.isnan(v_unscoped):
        return metric_scoped
    
    # Handle gt
    if gt_scoped and not gt_unscoped:
        return metric_unscoped
    if not gt_scoped and gt_unscoped:
        return metric_scoped

    if v_scoped > v_unscoped:
        return metric_unscoped
    if v_scoped < v_unscoped:
        return metric_scoped
    if v_scoped == v_unscoped:
        return None

In [28]:

dfs = []
for (domain, prob_name, planner, config), df in df_fd.groupby(["domain", "prob_name", "planner", "config"]):
    df = df.copy()
    df["metric"] = df["metric"].replace(fd_colmap)
    df = df.set_index("metric")
    # display(df)
    prob_num = re.search("\d+", prob_name).group()
    dom_prob = domain + " " + prob_num
    # df_new = pd.DataFrame({"Problem":dom_prob})
    # df_new.loc[]

        
    d = {
        "Problem": [dom_prob],
        "Config": [config]
    }
    # col2format = {}
    paired_metrics =  ["Generated Nodes", "Node Expansions", "Total Time", "Plan Time" ]
    for metric in paired_metrics:
        metric_scoped, metric_unscoped = get_scoped_unscoped_names(metric)
        d[metric_scoped] = [get_table_str(df, metric_scoped)]
        d[metric_unscoped] = [get_table_str(df, metric_unscoped)]
        better_column = get_better_col(df, metric)
        if better_column is not None:
            d[better_column] = ["\\mathbf{" + d[better_column][0] + "}"]

    solo_metrics = ["Translate", "Scoping"]
    for metric in solo_metrics:
        d[metric] = [get_table_str(df, metric)]

    metrics_all = solo_metrics
    for c in paired_metrics:
        metrics_all.extend(list(get_scoped_unscoped_names(c)))
        
    for c in metrics_all:
        d[c][0] = "$" + d[c][0] + "$"

    # [dom_prob, "Generated Nodes (Unscoped)", "Generated Nodes (Scoped)"]
    df_new = pd.DataFrame(data=d)
    # display(df_new)
    dfs.append(df_new)

df_out = pd.concat(dfs)
display(df_out)

,Problem,Config,Generated Nodes (Scoped),Generated Nodes (Unscoped),Node Expansions (Scoped),Node Expansions (Unscoped),Total Time (Scoped),Total Time (Unscoped),Plan Time (Scoped),Plan Time (Unscoped),Translate,Scoping
0,driverlog 15,lmcut,$\mathbf{21186.0 \pm 0.0}$,$22980.0 \pm 0.0$,$\mathbf{1379.0 \pm 0.0}$,$1392.0 \pm 0.0$,$7.17 \pm 0.23$,$\mathbf{4.9 \pm 0.16}$,$\mathbf{3.08 \pm 0.07}$,$4.44 \pm 0.15$,$0.46 \pm 0.02$,$3.63 \pm 0.15$
0,driverlog 15,ms,$\mathbf{7289831.0 \pm 0.0}$,$8796110.0 \pm 0.0$,$\mathbf{460244.0 \pm 0.0}$,$527636.0 \pm 0.0$,$13.51 \pm 0.54$,$\mathbf{12.85 \pm 0.5}$,$\mathbf{9.41 \pm 0.39}$,$12.4 \pm 0.48$,$0.46 \pm 0.01$,$3.64 \pm 0.18$
0,driverlog 16,lmcut,$\mathbf{60306.0 \pm 0.0}$,$87465.0 \pm 0.0$,$\mathbf{3087.0 \pm 0.0}$,$3618.0 \pm 0.0$,$\mathbf{17.08 \pm 0.54}$,$20.53 \pm 0.79$,$\mathbf{8.05 \pm 0.21}$,$19.84 \pm 0.78$,$0.7 \pm 0.02$,$8.33 \pm 0.34$
0,driverlog 16,ms,$\mathbf{577118.0 \pm 0.0}$,$925264.0 \pm 0.0$,$\mathbf{29618.0 \pm 0.0}$,$38681.0 \pm 0.0$,$13.58 \pm 0.61$,$\mathbf{8.28 \pm 0.27}$,$\mathbf{4.52 \pm 0.23}$,$7.58 \pm 0.25$,$0.69 \pm 0.02$,$8.37 \pm 0.39$
0,driverlog 17,lmcut,$\mathbf{21030.0 \pm 0.0}$,$28926.0 \pm 0.0$,$\mathbf{985.0 \pm 0.0}$,$1058.0 \pm 0.0$,$\mathbf{18.27 \pm 0.48}$,$22.8 \pm 0.96$,$\mathbf{7.87 \pm 0.24}$,$21.97 \pm 0.94$,$0.83 \pm 0.03$,$9.57 \pm 0.31$
0,driverlog 17,ms,$\mathbf{88705990.0 \pm 0.0}$,$198623900.0 \pm 0.0$,$\mathbf{4607258.0 \pm 0.0}$,$7768684.0 \pm 0.0$,$\mathbf{60.61 \pm 1.8}$,$154.87 \pm 4.87$,$\mathbf{50.25 \pm 1.53}$,$154.03 \pm 4.84$,$0.83 \pm 0.03$,$9.53 \pm 0.32$
0,logistics 15,lmcut,$\mathbf{117643.0 \pm 0.0}$,$153488.0 \pm 0.0$,$6395.0 \pm 0.0$,$6395.0 \pm 0.0$,$\mathbf{4.23 \pm 0.14}$,$11.92 \pm 0.19$,$\mathbf{3.29 \pm 0.07}$,$11.63 \pm 0.18$,$0.29 \pm 0.06$,$0.66 \pm 0.09$
0,logistics 15,ms,$\mathbf{11646320.0 \pm 0.0}$,$140607200.0 \pm 0.0$,$\mathbf{672736.0 \pm 0.0}$,$5951997.0 \pm 0.0$,$\mathbf{12.09 \pm 0.84}$,$73.99 \pm 2.94$,$\mathbf{11.14 \pm 0.8}$,$73.72 \pm 2.92$,$0.27 \pm 0.02$,$0.68 \pm 0.03$
0,logistics 20,lmcut,$\mathbf{260336.0 \pm 0.0}$,$381235.0 \pm 0.0$,$\mathbf{14390.0 \pm 0.0}$,$14798.0 \pm 0.0$,$\mathbf{6.86 \pm 0.18}$,$27.21 \pm 0.31$,$\mathbf{5.89 \pm 0.12}$,$26.93 \pm 0.29$,$0.28 \pm 0.02$,$0.69 \pm 0.05$
0,logistics 20,ms,$\mathbf{1524997.0 \pm 0.0}$,$6941424.0 \pm 0.0$,$\mathbf{86663.0 \pm 0.0}$,$289584.0 \pm 0.0$,$9.51 \pm 0.56$,$\mathbf{8.07 \pm 0.44}$,$8.55 \pm 0.51$,$\mathbf{7.79 \pm 0.42}$,$0.28 \pm 0.02$,$0.68 \pm 0.04$


In [35]:
def df2tex(df: pd.DataFrame) -> str:
    
    rows = ["\\begin{table*}[t]","\\resizebox{\\textwidth}{!}{%","\\begin{tabular}{" + "l"*len(df.columns) + "}"]
    rows.append("\\toprule")
    rows.append(" & ".join(df.columns) + "\\\\ \\midrule")
    for ind, r in df.iterrows():
        rows.append(" & ".join(list(r.values)) + "\\\\")
    rows.append("\\bottomrule")
    rows.append("\\end{tabular}")
    rows.append("}")
    rows += ["\\caption{Results for all of our experiments.}","\\label{table:experiment_times}", "\\end{table*}"]
    return "\n".join(rows)

In [36]:
s_out = df2tex(df_out)
with open("tmp.tex", "w") as f:
    f.write(s_out)

Problem (domain and prob name)
unscoped state and action space
scoped state and action space
scoping time
planning time (scoped)
total time (scoped)
planning time (unscoped)

planner and config

## SAS domain size calculator

In [ ]:
fd_logs_dir = "ignore/logs/fd/lmcut/examples/IPC_domains_propositional/zenotravel/prob10/0"
fd_domains = [x for x in  os.listdir(fd_logs_dir) if os.path.isdir(f"{fd_logs_dir}/{x}")]
for domain in fd_domains:
    probs = [x for x in os.listdir(f"{fd_logs_dir}/domain") if os.path.isdir(os.path.join(fd_logs_dir, domain, x))]
    for prob in probs:
        pth_scoped = os.path.join(fd_logs_dir, domain, prob, "0/unscoped2_scoped.sas")
        pth_unscoped = os.path.join(fd_logs_dir, domain, prob, "0/unscoped2.sas")
        